<a href="https://colab.research.google.com/github/Ali-Noghabi/face-mask-detection/blob/main/mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-Time Mask Detection

## Introduction

This project involves real-time mask detection using a pre-trained neural network. It can detect faces in video streams or webcam feeds and classify whether the person is wearing a mask or not. The project leverages deep learning techniques and computer vision to provide accurate and real-time mask detection.


## Project Setup

To run this project, you need to set up your environment and install the required libraries.

### Libraries Used

- **TensorFlow**: For loading and running the mask detection model.
- **OpenCV**: For real-time video processing and face detection.
- **imutils**: For basic image processing functions.
- **NumPy**: For numerical operations.

### Installation

To install the necessary libraries, run:

```bash
pip install tensorflow opencv-python imutils numpy
```

### Directory Structure

- `face_detector/deploy.prototxt`: Configuration file for the face detection model.
- `face_detector/res10_300x300_ssd_iter_140000.caffemodel`: Pre-trained weights for the face detection model.
- `mask_detector/mask_detector_model.keras`: Pre-trained mask detector model.
- `mask_detection.py`: Main script to run the mask detection.


In [ ]:
!pip install tensorflow opencv-python imutils numpy

## Detect and Predict

The core functionality of the project involves detecting faces in a video frame and predicting whether a mask is worn. This is achieved using the following functions:

### Face Detection

Faces are detected using a pre-trained Caffe model. The model configuration is loaded from `deploy.prototxt` and the weights are loaded from `res10_300x300_ssd_iter_140000.caffemodel`.

```python
prototxtPath = "face_detector/deploy.prototxt"
weightsPath = "face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
```


In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

### Mask Detection

The [mask detection model](mask_detector\README.md) is a pre-trained Keras model loaded from `mask_detector_model.keras`. The model predicts the presence of a mask in detected faces.

```python
maskNet = load_model("mask_detector/mask_detector_model.keras")
```


### Detection and Prediction Function

This function processes each video frame, detects faces, and predicts mask usage.


In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            if face.shape[0] == 0 or face.shape[1] == 0:
                continue

            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    return (locs, preds)

## Running the Project

To run the project, you can process either a video file or use a live webcam feed.

### Processing a Video File

Ensure that the video file is in the same directory as the script or provide the full path.

### Using the Webcam

To use the webcam for real-time mask detection, simply run the script without any arguments:

Press 'q' to exit the video display.


In [ ]:
# Load face detector model
prototxtPath = "face_detector/deploy.prototxt"
weightsPath = "face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# Load mask detector model
maskNet = load_model("mask_detector/mask_detector_model.keras")

In [ ]:
# Function to process video file
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (400, 400))

        (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

        for (box, pred) in zip(locs, preds):
            (startX, startY, endX, endY) = box
            (mask, withoutMask) = pred

            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

            cv2.putText(frame, label, (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        # Press 'q' to exit the video display
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage:
# Ensure that you have the video file in the same directory or provide the full path
video_path = 'video.mp4'
process_video(video_path)

In [ ]:
# Function to process the webcam feed
def process_webcam():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (400, 400))

        (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

        for (box, pred) in zip(locs, preds):
            (startX, startY, endX, endY) = box
            (mask, withoutMask) = pred

            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

            cv2.putText(frame, label, (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        # Press 'q' to exit the video display
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start live video processing from webcam
process_webcam()

## Results

The project will display the video feed with bounding boxes around detected faces. Each box will have a label indicating whether a mask is detected or not, along with the confidence level.

Example output:

![Mask](with_mask.png)
![NoMask](without_mask.png)


## References

- [TensorFlow](https://www.tensorflow.org/)
- [OpenCV](https://opencv.org/)
- [imutils](https://github.com/jrosebr1/imutils)
- [NumPy](https://numpy.org/)
